In [98]:
import os
import librosa
import numpy as np
import random

In [99]:
pip install numpy==1.24.3

5279.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [100]:
# Define the paths to your audio directories
koala_dir = '/Users/joanne/Desktop/koala_classification_project/koala/'
non_koala_dir = '/Users/joanne/Desktop/koala_classification_project/non_koala/'


In [ ]:
len(total_koala_data)

540

In [108]:
pip install torch torchvision torchaudio librosa numpy

5522.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install torch torchvision torchaudio librosa numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [114]:
import os
import librosa
import numpy as np

def list_audio_files(directory):
    return [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.wav')]

def load_audio_files(file_paths, sr=22050):
    return [librosa.load(file_path, sr=sr)[0] for file_path in file_paths]

In [217]:
def augment_audio(audio, sr=22050):
    # Example augmentations: add your own methods as needed
    augmented_audios = []
    
    # Time stretching
    augmented_audios.append(librosa.effects.time_stretch(audio, rate=0.9))
    augmented_audios.append(librosa.effects.time_stretch(audio, rate=1.1))
    
    # Pitch shifting
    augmented_audios.append(librosa.effects.pitch_shift(audio, sr=sr, n_steps=-2))
    augmented_audios.append(librosa.effects.pitch_shift(audio, sr=sr, n_steps=2))
    
    return augmented_audios

def pad_or_truncate(audio, target_length):
    if len(audio) < target_length:
        audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
    elif len(audio) > target_length:
        audio = audio[:target_length]
    return audio

def prepare_data(directory, target_length=16000, num_augmentations_per_sample=2):
    file_paths = list_audio_files(directory)
    data = load_audio_files(file_paths, sr=22050)
    
    augmented_data = []
    for audio in data:
        augmented_audios = augment_audio(audio, sr=22050)
        augmented_data.extend([pad_or_truncate(a, target_length) for a in augmented_audios[:num_augmentations_per_sample]])
    
    # Pad or truncate original data
    data = [pad_or_truncate(audio, target_length) for audio in data]

    return data + augmented_data

In [227]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import librosa
import torchvision.transforms as transforms
import os

def audio_to_melspectrogram(audio, sr=22050, n_mels=128, max_len=128):
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    if mel_spec_db.shape[1] < max_len:
        pad_width = max_len - mel_spec_db.shape[1]
        mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mel_spec_db = mel_spec_db[:, :max_len]
    return mel_spec_db

class AudioDataset(Dataset):
    def __init__(self, data, labels, transform=None, max_len=128):
        self.data = data
        self.labels = labels
        self.transform = transform
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio = self.data[idx]
        label = self.labels[idx]
        
        mel_spec_db = audio_to_melspectrogram(audio, max_len=self.max_len)
        
        # Add channel dimension: [n_mels, max_len] -> [1, n_mels, max_len]
        mel_spec_db = np.expand_dims(mel_spec_db, axis=0)  # Shape: [1, n_mels, max_len]
        
        # Convert to tensor
        mel_spec_db = torch.tensor(mel_spec_db, dtype=torch.float32)
        
        return mel_spec_db, label

# Define the transform
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts numpy array to torch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

# Example function to prepare data
def prepare_data(data_dir, target_length=16000, augment_audio_func=None, num_augmentations_per_sample=0):
    data = []
    for file in os.listdir(data_dir):
        if file.endswith(".wav"):
            file_path = os.path.join(data_dir, file)
            audio, _ = librosa.load(file_path, sr=22050)
            if len(audio) > target_length:
                audio = audio[:target_length]
            elif len(audio) < target_length:
                audio = np.pad(audio, (0, target_length - len(audio)), 'constant')

            data.append(audio)

            if augment_audio_func:
                for _ in range(num_augmentations_per_sample):
                    augmented_audio = augment_audio_func(audio)
                    data.append(augmented_audio)

    return np.array(data)

# Example augment function
def augment_audio(audio):
    return audio[::-1]  # Example: reverse the audio as a simple augmentation

# Load your data
koala_dir = "/Users/joanne/Desktop/koala_classification_project/koala/"
non_koala_dir = "/Users/joanne/Desktop/koala_classification_project/non_koala/"
koala_data = prepare_data(koala_dir, target_length=16000, augment_audio_func=augment_audio, num_augmentations_per_sample=2)
non_koala_data = prepare_data(non_koala_dir, target_length=16000)

# Check the data lengths
print(f'Number of koala samples: {len(koala_data)}')
print(f'Number of non-koala samples: {len(non_koala_data)}')

# Ensure you have 540 koala samples and 560 non-koala samples
assert len(koala_data) == 540, "Koala data does not have 540 samples"
assert len(non_koala_data) == 560, "Non-koala data does not have 560 samples"

koala_labels = [1] * len(koala_data)
non_koala_labels = [0] * len(non_koala_data)

# Combine the datasets
data = np.concatenate((koala_data, non_koala_data), axis=0)
labels = np.array(koala_labels + non_koala_labels)

# Create the dataset and dataloader
train_dataset = AudioDataset(data, labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Check the batch shape
for inputs, labels in train_loader:
    print("Batch shape:", inputs.shape)  # Should be [batch_size, channels, height, width]
    break

Number of koala samples: 540
Number of non-koala samples: 560
Batch shape: torch.Size([32, 1, 128, 128])


In [231]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import librosa
import torchvision.transforms as transforms
import os

def audio_to_melspectrogram(audio, sr=22050, n_mels=128, max_len=128):
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    if mel_spec_db.shape[1] < max_len:
        pad_width = max_len - mel_spec_db.shape[1]
        mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mel_spec_db = mel_spec_db[:, :max_len]
    return mel_spec_db

class AudioDataset(Dataset):
    def __init__(self, data, labels, transform=None, max_len=128):
        self.data = data
        self.labels = labels
        self.transform = transform
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio = self.data[idx]
        label = self.labels[idx]
        
        mel_spec_db = audio_to_melspectrogram(audio, max_len=self.max_len)
        
        # Add channel dimension: [n_mels, max_len] -> [1, n_mels, max_len]
        mel_spec_db = np.expand_dims(mel_spec_db, axis=0)  # Shape: [1, n_mels, max_len]
        
        # Convert to tensor
        mel_spec_db = torch.tensor(mel_spec_db, dtype=torch.float32)
        
        return mel_spec_db, label

# Define the transform
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts numpy array to torch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

# Example function to prepare data
def prepare_data(data_dir, target_length=16000, augment_audio_func=None, num_augmentations_per_sample=0):
    data = []
    for file in os.listdir(data_dir):
        if file.endswith(".wav"):
            file_path = os.path.join(data_dir, file)
            audio, _ = librosa.load(file_path, sr=22050)
            if len(audio) > target_length:
                audio = audio[:target_length]
            elif len(audio) < target_length:
                audio = np.pad(audio, (0, target_length - len(audio)), 'constant')

            data.append(audio)

            if augment_audio_func:
                for _ in range(num_augmentations_per_sample):
                    augmented_audio = augment_audio_func(audio)
                    data.append(augmented_audio)

    return np.array(data)

# Example augment function
def augment_audio(audio):
    return audio[::-1]  # Example: reverse the audio as a simple augmentation

# Load your data
koala_dir = "/Users/joanne/Desktop/koala_classification_project/koala/"
non_koala_dir = "/Users/joanne/Desktop/koala_classification_project/non_koala/"
koala_data = prepare_data(koala_dir, target_length=16000, augment_audio_func=augment_audio, num_augmentations_per_sample=2)
non_koala_data = prepare_data(non_koala_dir, target_length=16000)

# Check the data lengths
print(f'Number of koala samples: {len(koala_data)}')
print(f'Number of non-koala samples: {len(non_koala_data)}')

# Ensure you have 540 koala samples and 560 non-koala samples
assert len(koala_data) == 540, "Koala data does not have 540 samples"
assert len(non_koala_data) == 560, "Non-koala data does not have 560 samples"

koala_labels = [1] * len(koala_data)
non_koala_labels = [0] * len(non_koala_data)

# Combine the datasets
data = np.concatenate((koala_data, non_koala_data), axis=0)
labels = np.array(koala_labels + non_koala_labels)

# Create the dataset
dataset = AudioDataset(data, labels, transform=transform)

Number of koala samples: 540
Number of non-koala samples: 560


In [229]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetAudioClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetAudioClassifier, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet.fc = nn.Linear(in_features=self.resnet.fc.in_features, out_features=num_classes)

    def forward(self, x):
        return self.resnet(x)

In [233]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import KFold

# Updated ResNetAudioClassifier with weights argument
class ResNetAudioClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetAudioClassifier, self).__init__()
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet.fc = nn.Linear(in_features=self.resnet.fc.in_features, out_features=num_classes)

    def forward(self, x):
        return self.resnet(x)

# Assume you have the AudioDataset class and data preparation functions defined as before

def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    return running_loss / len(train_loader)

def validate_model(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return running_loss / len(val_loader), accuracy

def cross_validate_model(dataset, k_folds=10, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kfold = KFold(n_splits=k_folds, shuffle=True)
    
    results = {}
    
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold+1}')
        print('--------------------------------')
        
        train_subsampler = Subset(dataset, train_ids)
        val_subsampler = Subset(dataset, val_ids)
        
        train_loader = DataLoader(train_subsampler, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_subsampler, batch_size=32, shuffle=False)
        
        model = ResNetAudioClassifier(num_classes=2).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        for epoch in range(num_epochs):
            train_loss = train_model(model, train_loader, criterion, optimizer, device)
            val_loss, val_accuracy = validate_model(model, val_loader, criterion, device)
            
            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        
        results[fold] = val_accuracy
    
    print('--------------------------------')
    print('K-FOLD CROSS VALIDATION RESULTS')
    print('--------------------------------')
    for key, value in results.items():
        print(f'Fold {key+1}: {value:.2f}%')
    avg_accuracy = np.mean(list(results.values()))
    print(f'Average Accuracy: {avg_accuracy:.2f}%')

# Perform cross-validation
cross_validate_model(train_dataset, k_folds=10, num_epochs=10)

FOLD 1
--------------------------------
Epoch 1/10, Train Loss: 0.3562, Val Loss: 2.9488, Val Accuracy: 50.91%
Epoch 2/10, Train Loss: 0.1852, Val Loss: 0.5636, Val Accuracy: 68.18%
Epoch 3/10, Train Loss: 0.1714, Val Loss: 0.8391, Val Accuracy: 81.82%
Epoch 4/10, Train Loss: 0.1645, Val Loss: 0.2544, Val Accuracy: 90.00%
Epoch 5/10, Train Loss: 0.1707, Val Loss: 0.2153, Val Accuracy: 89.09%
Epoch 6/10, Train Loss: 0.1379, Val Loss: 3.2615, Val Accuracy: 47.27%
Epoch 7/10, Train Loss: 0.1852, Val Loss: 0.5290, Val Accuracy: 77.27%
Epoch 8/10, Train Loss: 0.1267, Val Loss: 0.9463, Val Accuracy: 77.27%
Epoch 9/10, Train Loss: 0.1409, Val Loss: 0.6863, Val Accuracy: 66.36%
Epoch 10/10, Train Loss: 0.1239, Val Loss: 2.5030, Val Accuracy: 48.18%
FOLD 2
--------------------------------
Epoch 1/10, Train Loss: 0.3981, Val Loss: 0.6324, Val Accuracy: 83.64%
Epoch 2/10, Train Loss: 0.2002, Val Loss: 0.8059, Val Accuracy: 70.00%
Epoch 3/10, Train Loss: 0.2106, Val Loss: 0.2704, Val Accuracy: 90.

In [60]:
pip install torch torchvision torchaudio librosa

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 61.0 MB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.2 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 9.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
